# Feature postprocessing from paper.
- Title:  "Predicting yeast synthetic lethal genetic interactions using protein domains"
- Authors: Bo Li, Feng Luo,School of Computing,Clemson University,Clemson, SC, USA
- e-mail: bol, luofeng@clemson.edu
- year:2009

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict 
import seaborn as sns
import matplotlib.cm as cm
import scipy as scipy
import random

### Importing datasets 

#### Link to the github repo where the datasets to be downloaded:
[DOWNLOAD THE DATASETS HERE](https://github.com/leilaicruz/machine-learning-for-yeast/tree/dev_Leila/datasets-for-learning)

In [2]:
import os
script_dir = os.path.dirname('__file__') #<-- absolute dir the script is in
rel_path_SL = "datasets/data-synthetic-lethals.xlsx"
rel_path_nSL="datasets/data-positive-genetic.xlsx"
rel_path_domains="datasets/proteins-domains-from-Pfam.xlsx"

abs_file_path_SL = os.path.join(script_dir, rel_path_SL)
abs_file_path_nSL = os.path.join(script_dir, rel_path_nSL)
abs_file_path_domains = os.path.join(script_dir, rel_path_domains)

# os.chdir('mini_book/docs/') #<-- for binder os.chdir('../')
# os.chdir('../')
my_path_sl= abs_file_path_SL
my_path_non_sl=abs_file_path_nSL
my_path_domains=abs_file_path_domains

data_sl=pd.read_excel(my_path_sl,header=0)
data_domains=pd.read_excel(my_path_domains,header=0,index_col='Unnamed: 0')
data_domains=data_domains.dropna()
data_nonsl=pd.read_excel(my_path_non_sl,header=0)

## Building the feature matrix
One matrix for true SL where each row is one pair of SL. Every raw will be a vector of 0,1 or 2 depending on the comparison with the domain list. For row i the jth element = 0 if the jth element of the domain list is not in neither protein A and B, 1, if it is in one of them and 2 if it is in both of them .

### Building the list of proteins domains id per protein pair separately :
- List of protein A: Search for the Sl/nSL database the query gene name and look in the protein domain database which protein domains id has each of those queries.
- List of protein B: Search for the Sl/nSL database the target gene name of the previous query and look in the protein domain database which protein domains id has each of those target genes.

In [3]:
# Selecting the meaningful columns in the respective dataset
domain_id_list=data_domains['domain-name']
query_gene=data_sl['gene-query-name']
target_gene=data_sl['gene-target-name']
query_gene_nonlethal=data_nonsl['gene-query-name']
target_gene_nonlethal=data_nonsl['gene-target-name']



# Initialising the arrays
protein_a_list=[]
protein_b_list=[]
protein_a_list_non=[]
protein_b_list_non=[]

population = np.arange(0,len(data_sl))

# For loop for 10000 pairs sampled randomly from the SL/nSl pair list , and creating a big array of proteind domains id per protein pair

for m in random.sample(list(population), 500):
    protein_a=data_domains[data_domains['name']==query_gene[m]]
    protein_b=data_domains[data_domains['name']==target_gene[m]]
    protein_a_list.append(protein_a['domain-name'].tolist())
    protein_b_list.append(protein_b['domain-name'].tolist())

    protein_a_non=data_domains[data_domains['name']==query_gene_nonlethal[m]]
    protein_b_non=data_domains[data_domains['name']==target_gene_nonlethal[m]]
    protein_a_list_non.append(protein_a_non['domain-name'].tolist())
    protein_b_list_non.append(protein_b_non['domain-name'].tolist())

In [4]:
print('We are going to analyze',len((protein_a_list)) ,'protein pairs, out of',len(data_sl),'SL protein pairs')
print('We are going to analyze',len((protein_a_list_non)) ,'protein pairs, out of',len(data_nonsl),'positive protein pairs')

We are going to analyze 500 protein pairs, out of 17871 SL protein pairs
We are going to analyze 500 protein pairs, out of 43340 positive protein pairs


### Postprocessing #1:  Remove protein pairs from study if either protein in the pair does not contain any domain

In [5]:
def remove_empty_domains(protein_list_search,protein_list_pair):
    index=[]
    for i in np.arange(0,len(protein_list_search)):
        if protein_list_search[i]==[] or protein_list_pair[i]==[]:
            index.append(i) ## index of empty values for the protein_a_list meaning they dont have any annotated domain

    y=[x for x in np.arange(0,len(protein_list_search)) if x not in index] # a list with non empty values from protein_a list

    protein_list_search_new=[]
    protein_list_pair_new=[]
    for i in y:
        protein_list_search_new.append(protein_list_search[i])
        protein_list_pair_new.append(protein_list_pair[i])
    return protein_list_search_new,protein_list_pair_new

## evaluating the function

protein_a_list_new,protein_b_list_new=remove_empty_domains(protein_a_list,protein_b_list)

protein_a_list_non_new,protein_b_list_non_new=remove_empty_domains(protein_a_list_non,protein_b_list_non)

In [6]:
print('The empty domain in the SL were:', len(protein_a_list)-len(protein_a_list_new), 'out of', len(protein_a_list),'domains')
print('The empty domain in the nSL were:', len(protein_a_list_non)-len(protein_a_list_non_new), 'out of', len(protein_a_list_non),'domains')

The empty domain in the SL were: 85 out of 500 domains
The empty domain in the nSL were: 87 out of 500 domains


### Feature engineering: Select from each ordered indexes of domain id list which of them appear once, in both or in any of the domains of each protein pair

#### Define function `get_indexes`

In [7]:
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y] # a function that give the index of whether a value appear in array or not
a=[1,2,2,4,5,6,7,8,9,10]
get_indexes(2,a)

[1, 2]

In [8]:
def feature_building(protein_a_list_new,protein_b_list_new):
    x = np.unique(domain_id_list)
    ## To avoid taking repeated domains from one protein of the pairs , lets reduced the domains of each protein from the pairs to their unique members
    protein_a_list_unique=[]
    protein_b_list_unique=[]
    for i in np.arange(0,len(protein_a_list_new)):
        protein_a_list_unique.append(np.unique(protein_a_list_new[i]))
        protein_b_list_unique.append(np.unique(protein_b_list_new[i]))
        
    protein_feat_true=np.zeros(shape=(len(x),len(protein_a_list_unique)))
    pair_a_b_array=[]
    for i in np.arange(0,len(protein_a_list_unique)):
        index_a=[]
        
        pair=[protein_a_list_unique[i],protein_b_list_unique[i]]
        pair_a_b=np.concatenate(pair).ravel()
        pair_a_b_array.append(pair_a_b)

    j=0
    for i in pair_a_b_array:  
        array,index,counts=np.unique(i,return_index=True,return_counts=True)
        
        for k,m in zip(counts,array):
            if k ==2:
                protein_feat_true[get_indexes(m,x),j]=2
                
            if k==1:
                protein_feat_true[get_indexes(m,x),j]=1
        j=j+1
    return protein_feat_true

In [9]:
protein_feat_true=feature_building(protein_b_list_new=protein_b_list_new,protein_a_list_new=protein_a_list_new)
protein_feat_true_pd=pd.DataFrame(protein_feat_true.T)

protein_feat_non_true=feature_building(protein_b_list_new=protein_b_list_non_new,protein_a_list_new=protein_a_list_non_new)
protein_feat_non_true_pd=pd.DataFrame(protein_feat_non_true.T)

### How many ones and twos are in each dataset

In [10]:
index_2_true=protein_feat_true_pd.where(protein_feat_true_pd==2)
index_2_true_count=index_2_true.count(axis=1).sum()

index_1_true=protein_feat_true_pd.where(protein_feat_true_pd==1)
index_1_true_count=index_1_true.count(axis=1).sum()

index_2_nontrue=protein_feat_non_true_pd.where(protein_feat_non_true_pd==2)
index_2_nontrue_count=index_2_nontrue.count(axis=1).sum()

index_1_nontrue=protein_feat_non_true_pd.where(protein_feat_non_true_pd==1)
index_1_nontrue_count=index_1_nontrue.count(axis=1).sum()


print('fraction of twos in the SL array is',index_2_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)))
print('fraction of ones in the SL array is',index_1_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)))
print('fraction of twos in the PI array is',index_2_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)))
print('fraction of ones in the PI array is',index_1_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)))

fraction of twos in the SL array is 2.867669023200239e-05
fraction of ones in the SL array is 0.0009949218361047496
fraction of twos in the PI array is 6.403457867248314e-06
fraction of ones in the PI array is 0.0009156944750165089


#### Bar plot to visualize these numbers

In [11]:
plt.bar(['fraction of 2 in the nSL','fraction of 1 in the nSL'],[index_2_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns)),index_1_nontrue_count/(len(protein_feat_non_true_pd.index)*len(protein_feat_non_true_pd.columns))],alpha=0.6,color=['blue','lightblue']), 

plt.bar(['fraction of 2 in SL ','fraction of 1 in SL'],[index_2_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns)),index_1_true_count/(len(protein_feat_true_pd.index)*len(protein_feat_true_pd.columns))],alpha=0.6,color=['coral','lightcoral'])

plt.ylabel('Fraction from the population')
plt.yscale('log')
plt.xticks(rotation=40)

([0, 1, 2, 3],
 [Text(0, 0, ''), Text(0, 0, ''), Text(0, 0, ''), Text(0, 0, '')])

#### Adding the labels(response variables) to each dataset

In [12]:
protein_feat_true_pd['lethality']=np.ones(shape=(len(protein_a_list_new)))
protein_feat_non_true_pd['lethality']=np.zeros(shape=(len(protein_a_list_non_new)))

#### Joining both datasets

In [13]:
feature_post=pd.concat([protein_feat_true_pd,protein_feat_non_true_pd],axis=0)
feature_post=feature_post.set_index(np.arange(0,len(protein_a_list_new)+len(protein_a_list_non_new)))
print('The number of features are:',feature_post.shape[1])
print('The number of samples are:',feature_post.shape[0])

The number of features are: 3026
The number of samples are: 828


### Postprocessing and exploration of the feature matrix of both datasets

In [14]:
mean=feature_post.T.describe().loc['mean']
std=feature_post.T.describe().loc['std']
lethality=feature_post['lethality']

corr_keys=pd.concat([mean,std,lethality],axis=1)

### Viz of the stats

In [15]:
pair=sns.pairplot(corr_keys,hue='lethality',diag_kind='kde',kind='reg',palette='colorblind')
pair.fig.suptitle('Pairplot to see data dependencies with Lethality',y=1.08)

Text(0.5, 1.08, 'Pairplot to see data dependencies with Lethality')

In [16]:
X, y = feature_post.drop(columns=["lethality"]), feature_post["lethality"]

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_sl=protein_feat_true_pd.drop(columns=['lethality'])
x_nsl=protein_feat_non_true_pd.drop(columns=['lethality'])

X_sl_scaled = scaler.fit_transform(x_sl)
X_nsl_scaled = scaler.fit_transform(x_nsl)

In [18]:
np.shape(X_sl_scaled)

(415, 3025)

## How redundant are each of the protein domains?

In [19]:
def PCA_component_contribution(scaled_matrix,original_data):
    from sklearn.decomposition import PCA

    model = PCA(0.95).fit(scaled_matrix)
    ## apply dimensionality reduction to X_train
    output_pca = model.transform(scaled_matrix)
    
    total=sum(model.explained_variance_)


   
    # number of components , that it will be the number of main axes times the number of original components 
    n_pcs= model.components_.shape[0] # the amount of non redundant protein domains 

    # get the index of the most important feature on EACH component
    # LIST COMPREHENSION HERE
    most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

    initial_feature_names = original_data.columns
    # get the names
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
    

    # LIST COMPREHENSION HERE AGAIN
    dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

    # build the dataframe
    df = pd.DataFrame(dic.items(),columns=['pca-component','domain-number'])

    return df,model.components_

In [20]:
df_sl,components_pca_nsl=PCA_component_contribution(X_nsl_scaled,x_nsl)
df_nsl,components_pca_sl=PCA_component_contribution(X_sl_scaled,x_sl)

In [21]:
df_sl.head()

pca-component  domain-number
0           PC0           1028
1           PC1            221
2           PC2             76
3           PC3            135
4           PC4            262

## which are the shared domains between nSL and SL pairs , in the PCA reduced samples

In [22]:
df_shared_domains=pd.merge(df_sl,df_nsl,how='inner',on='domain-number')
domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_shared_domains['domain-description']=domains_description[df_shared_domains['domain-number']]


df_shared_domains

pca-component_x  domain-number pca-component_y  \
0              PC0           1028             PC0   
1              PC8              3           PC153   
2              PC8              3           PC159   
3             PC20           2544            PC18   
4             PC21           2113             PC4   
5             PC22            369            PC34   
6             PC25            274            PC17   
7             PC27           1428           PC211   
8             PC27           1428           PC251   
9             PC34           2610            PC21   
10            PC44           1174           PC247   
11            PC49           1887            PC65   
12            PC58            773            PC75   
13            PC58            773            PC76   
14            PC58            773            PC77   
15            PC62           2150           PC178   
16            PC70           1044            PC53   
17            PC70           1044            PC57   
18            PC73           1350           PC219   
19            PC74            109            PC87   
20            PC74            109            PC88   
21            PC76            625           PC188   
22            PC81            892            PC25   
23            PC84            176            PC92   
24            PC84            176            PC98   
25            PC98           1903            PC23   
26           PC114           1114           PC249   
27           PC115            486           PC225   
28           PC128            486           PC225   
29           PC130           1325           PC122   
30           PC130           1325           PC123   
31           PC142            363            PC39   
32           PC153           1347           PC210   
33           PC153           1347           PC240   
34           PC156             68           PC125   
35           PC157           1977           PC233   
36           PC172            450           PC256   
37           PC186             39           PC135   
38           PC187             39           PC135   
39           PC189             39           PC135   
40           PC191           1168           PC229   
41           PC191           1168           PC238   
42           PC191           1168           PC239   
43           PC197           1168           PC229   
44           PC197           1168           PC238   
45           PC197           1168           PC239   
46           PC193           2133           PC117   
47           PC209           1046           PC110   
48           PC218           2025           PC142   
49           PC218           2025           PC260   
50           PC219            665           PC230   
51           PC219            665           PC246   
52           PC226           1612           PC179   
53           PC230            494           PC204   
54           PC234           2127           PC130   
55           PC236           1250            PC80   
56           PC244           1924            PC10   
57           PC272           2584            PC84   
58           PC277           1260           PC118   

                                   domain-description  
0                 GatB_Yqey; Asn/Gln amidotransferase  
1   2-Hacid_dh_C; D-isomer specific 2-hydroxyacid ...  
2   2-Hacid_dh_C; D-isomer specific 2-hydroxyacid ...  
3   Sybindin; Trafficking protein particle complex...  
4       Rav1p_C; RAVE complex protein Rav1 C-terminal  
5                   CNH; Citron homology (CNH) domain  
6                     BOP1NT; BOP1, N-terminal domain  
7         MatE; Multi antimicrobial extrusion protein  
8         MatE; Multi antimicrobial extrusion protein  
9   TPK_B1_binding; Thiamin pyrophosphokinase, vit...  
10               His_biosynth; Histidine biosynthesis  
11        Peptidase_S24; Peptidase S24/S26A/S26B/S26C  
12         Dynein_light; Dynein light chain, type 1/2  
13         Dynein_light; Dynein light 

In [23]:
print('The number of shared domains between SL and nSL pairs,after PCA, is = ',len(df_shared_domains), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_shared_domains)/(len(df_sl)+len(df_nsl)),'%')

The number of shared domains between SL and nSL pairs,after PCA, is =  59 out of 556 so the 10.611510791366907 %


## Domains exclusive to SL

In [24]:
df_sl_exclusive=pd.merge(df_sl,df_nsl,how='left',on='domain-number')

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_sl_exclusive['domain-description']=domains_description[df_sl_exclusive['domain-number']]
df_sl_exclusive[pd.isnull(df_sl_exclusive['pca-component_y'])]

pca-component_x  domain-number pca-component_y  \
1               PC1            221             NaN   
2               PC2             76             NaN   
3               PC3            135             NaN   
4               PC4            262             NaN   
5               PC5             93             NaN   
..              ...            ...             ...   
293           PC278            337             NaN   
294           PC279            282             NaN   
295           PC280           1294             NaN   
296           PC281           1462             NaN   
297           PC282           1996             NaN   

                                    domain-description  
1    Apq12; Nuclear pore assembly and biogenesis pr...  
2    ALS_ss_C; Acetolactate synthase, small subunit...  
3    ATP-synt_ab_Xtn; ATPsynthase alpha/beta subuni...  
4    BCDHK_Adom3; Branched-chain alpha-ketoacid deh...  
5                  APG12; Ubiquitin-like protein Atg12  
..                                                 ...  
293               CDC48_N; CDC48, N-terminal subdomain  
294                    BRE1; E3 ubiquitin ligase, BRE1  
295                LIAS_N; Lipoyl synthase, N-terminal  
296  Met_10; SAM-dependent methyltransferase TRM5/T...  
297    RNA_Me_trans; RNA methyltransferase TK0422/Sfm1  

[239 rows x 4 columns]

In [25]:
print('The number of exclusive domains from SL pairs, after PCA, is = ',len(df_sl_exclusive), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_sl_exclusive)/(len(df_sl)+len(df_nsl)),'%')

The number of exclusive domains from SL pairs, after PCA, is =  298 out of 556 so the 53.597122302158276 %


In [26]:
df_nsl_exclusive=pd.merge(df_sl,df_nsl,how='right',on='domain-number')

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

df_nsl_exclusive['domain-description']=domains_description[df_nsl_exclusive['domain-number']]
df_nsl_exclusive[pd.isnull(df_nsl_exclusive['pca-component_x'])]

pca-component_x  domain-number pca-component_y  \
1               NaN            305             PC1   
2               NaN           1100             PC2   
3               NaN            602             PC3   
5               NaN             58             PC5   
6               NaN            195             PC6   
..              ...            ...             ...   
274             NaN           2478           PC264   
275             NaN             38           PC265   
276             NaN           1311           PC267   
277             NaN           2696           PC271   
278             NaN           2032           PC272   

                                    domain-description  
1                      Brr6_like_C_C; Brl1/Brr6 domain  
2                HBS1_N; HBS1-like protein, N-terminal  
3    DNA_RNApol_7kD; RNA polymerase archaeal subuni...  
5                    AEP1; ATPase expression protein 1  
6    Alpha-amylase; Glycosyl hydrolase, family 13, ...  
..                                                 ...  
274  Sin3_corepress; Histone deacetylase interactin...  
275              ABC2_membrane; ABC-2 type transporter  
276                Lactamase_B; Metallo-beta-lactamase  
277         Transketolase_N; Transketolase, N-terminal  
278  RNA_pol_Rpc4; DNA-directed RNA polymerase III ...  

[220 rows x 4 columns]

In [27]:
print('The number of exclusive domains from non SL pairs, after PCA, is = ',len(df_nsl_exclusive), 'out of', len(df_sl)+len(df_nsl), 'so the',100*len(df_nsl_exclusive)/(len(df_sl)+len(df_nsl)),'%')

The number of exclusive domains from non SL pairs, after PCA, is =  279 out of 556 so the 50.17985611510792 %


## Let see to the domains-number that get repeated after the PCA analysis , and selecting the most explanatory feature by its explained variance.

In [28]:
duplicated_features_sl=df_sl[df_sl.iloc[:,1].duplicated()]
repeated_features_sl=pd.DataFrame()

domains_name=np.unique(data_domains['domain-name'])
domains_description=np.unique(data_domains['domain-descrip'])

repeated_features_sl['domain-name']=domains_name[duplicated_features_sl.iloc[:,1]] 
repeated_features_sl['domain-description']=domains_description[duplicated_features_sl.iloc[:,1]]

duplicated_features_nsl=df_nsl[df_nsl.iloc[:,1].duplicated()]
repeated_features_nsl=pd.DataFrame()

repeated_features_nsl['domain-name']=domains_name[duplicated_features_nsl.iloc[:,1]] 
repeated_features_nsl['domain-description']=domains_description[duplicated_features_nsl.iloc[:,1]]

In [29]:
only_sl_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='left',on=['domain-name'])
data_only_sl = only_sl_pd[pd.isnull(only_sl_pd['domain-description_y'])]
data_only_sl['domain-description']=data_only_sl['domain-description_x']
data_only_sl.shape,df_sl_exclusive.shape

/usr/share/miniconda3/envs/wintest/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


((33, 4), (298, 4))

In [30]:
only_nsl_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='right',on=['domain-name'])
data_only_nsl = only_nsl_pd[pd.isnull(only_nsl_pd['domain-description_x'])]
data_only_nsl

domain-name domain-description_x  \
0      PF04423                  NaN   
1      PF07569                  NaN   
2      PF00782                  NaN   
3      PF00183                  NaN   
4      PF00650                  NaN   
5      PF07529                  NaN   
6      PF03105                  NaN   
7      PF04851                  NaN   
8      PF08647                  NaN   
9      PF00339                  NaN   
10     PF01909                  NaN   
11     PF01909                  NaN   
12     PF00080                  NaN   
13     PF00227                  NaN   
14     PF00653                  NaN   
15     PF00349                  NaN   
16     PF00633                  NaN   
17     PF01222                  NaN   
18     PF03874                  NaN   
19     PF13812                  NaN   
20     PF00022                  NaN   
21     PF07535                  NaN   
22     PF05557                  NaN   
23     PF01853                  NaN   
24     PF04191                  NaN   
25     PF00134                  NaN   
26     PF04280                  NaN   
27     PF03215                  NaN   
28     PF03215                  NaN   
29     PF00009                  NaN   
30     PF00400                  NaN   
31     PF00071                  NaN   
32     PF06839                  NaN   
33     PF08214                  NaN   
34     PF09135                  NaN   
35     PF06732                  NaN   
36     PF00899                  NaN   
37     PF03399                  NaN   
38     PF04939                  NaN   
39     PF04939                  NaN   
40     PF04939                  NaN   
41     PF03142                  NaN   
42     PF03142                  NaN   
43     PF05841                  NaN   
44     PF05841                  NaN   
45     PF05841                  NaN   
46     PF01199                  NaN   
47     PF11704                  NaN   
50     PF04420                  NaN   
51     PF01553                  NaN   
52     PF08654                  NaN   
53     PF04818                  NaN   
54     PF12171                  NaN   
55     PF01151                  NaN   
56     PF08690                  NaN   
57     PF02902                  NaN   
58     PF17097                  NaN   
59     PF11778                  NaN   
60     PF03366                  NaN   
61     PF09346                  NaN   

                                 domain-description_y  
0                               MARVEL; Marvel domain  
1             PFU; PLAA family ubiquitin binding, PFU  
2                                                COG6  
3        ANAPC5; Anaphase-promoting complex subunit 5  
4   CBFB_NFYA; Nuclear transcription factor Y subu...  
5                         PDT; Prephenate dehydratase  
6       Glyco_hydro_16; Glycoside hydrolase family 16  
7            Med13_C; Mediator complex, subunit Med13  
8   RNA_pol_L_2; DNA-directed RNA polymerase, RBP1...  
9   AdenylateSensor; AMPK, C-terminal adenylate se...  
10         Dynein_light; Dynein light chain, type 1/2  
11         Dynein_light; Dynein light chain, type 1/2  
12   AATF-Che1; AATF leucine zipper-containing domain  
13                ATG13; Autophagy-related protein 13  
14  CBFD_NFYB_HMF; Transcription factor CBF/NF-Y/a...  
15  Aft1_OSA; Transcription factor Aft1, osmotic s...  
16                            CAF1; Ribonuclease CAF1  
17               DASH_Dad2; DASH complex subunit Dad2  
18          Hydantoinase_A; Hydantoinase/oxoprolinase  
19   Tom6; Mitochondrial import receptor subunit Tom6  
20  40S_S4_C; 40S ribosomal protein S4, C-terminal...  
21                             PDZ_1; PDZ-like domain  
22                          NIF; FCP1 homology domain  
23                         Dopey_N; Dopey, N-terminal  
24  Lgl_C; Lethal giant larvae (Lgl)-like, C-termi...  
25                                               AGA2  
26                                LsmAD; LsmAD domain  
27  Glyoxalase; Glyoxalase/fosfomycin resistance/d...  

In [31]:
shared_domains_pd=pd.merge(repeated_features_sl,repeated_features_nsl,how='inner',on=['domain-name'])
shared_domains_pd


domain-name               domain-description_x  \
0     PF03770  Hira; TUP1-like enhancer of split   
1     PF03770  Hira; TUP1-like enhancer of split   

                domain-description_y  
0  Hira; TUP1-like enhancer of split  
1  Hira; TUP1-like enhancer of split

In [32]:
a_sl=x_sl.iloc[:,np.sort(df_sl.iloc[:,1])]
a_sl.describe().loc['mean'].hist(bins=8),
a_sl.describe().loc['std'].hist(bins=8,alpha=0.4)
plt.xlim([0,0.2])

(0.0, 0.2)

In [33]:
a_nsl=x_nsl.iloc[:,np.sort(df_nsl.iloc[:,1])]
a_nsl.describe().loc['mean'].hist(bins=8),
a_nsl.describe().loc['std'].hist(bins=8,alpha=0.4)
plt.xlim([0,0.2])

(0.0, 0.2)

### Proof of concept with *BEM1*

- Gather the domains of SL and nSL pairs from BEM1
- Check if the domains for the SL and nSL pairs are inside the most important domains after PCA.

In [34]:
domains_bem1=data_domains[data_domains['name']=='BEM1']['domain-descrip']
sl_bem1=data_sl[data_sl['gene-query-name']=='BEM1']['gene-target-name']

In [35]:
sl_bem1

2846      BEM2
2847     CDC24
5004      MSB1
13777     SMI1
13778     BNI1
13779     BEM2
14385     SKN7
Name: gene-target-name, dtype: object

In [36]:
data_sl[data_sl['gene-query-name']=='BEM1']

gene-query gene-target gene-query-name gene-target-name  \
2846     YBR200W     YER155C            BEM1             BEM2   
2847     YBR200W     YAL041W            BEM1            CDC24   
5004     YBR200W     YOR188W            BEM1             MSB1   
13777    YBR200W     YGR229C            BEM1             SMI1   
13778    YBR200W     YNL271C            BEM1             BNI1   
13779    YBR200W     YER155C            BEM1             BEM2   
14385    YBR200W     YHR206W            BEM1             SKN7   

                                  gene-query-description  \
2846   SRO1|phosphatidylinositol-3-phosphate-binding ...   
2847   SRO1|phosphatidylinositol-3-phosphate-binding ...   
5004   SRO1|phosphatidylinositol-3-phosphate-binding ...   
13777  SRO1|phosphatidylinositol-3-phosphate-binding ...   
13778  SRO1|phosphatidylinositol-3-phosphate-binding ...   
13779  SRO1|phosphatidylinositol-3-phosphate-binding ...   
14385  SRO1|phosphatidylinositol-3-phosphate-binding ...   

                                 gene-target-description     interaction-type  \
2846                           IPL2|SUP9|TSL1|L000000168  Synthetic Lethality   
2847   CLS4|Rho family guanine nucleotide exchange fa...  Synthetic Lethality   
5004                                          L000001184  Synthetic Lethality   
13777                                    KNR4|L000000909  Synthetic Lethality   
13778                   PPF3|SHE5|formin BNI1|L000000190  Synthetic Lethality   
13779                          IPL2|SUP9|TSL1|L000000168  Synthetic Lethality   
14385  BRY1|POS9|kinase-regulated stress-responsive t...     Dosage Lethality   

            paper-source  
2846   Peterson J (1994)  
2847   Peterson J (1994)  
5004     Bender A (1991)  
13777   Gorelik M (2011)  
13778   Gorelik M (2011)  
13779   Gorelik M (2011)  
14385   Bouquin N (1999)